In [ ]:
!pip install gcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602926 sha256=bc492614bcb68cd0c95711069f1b46b9fbbe6ca4c6ab68891c925a9fd7711cca
  Stored in directory: /root/.cache/pip/wheels/7c/30/88/5017af921da3a33af785f0d0fd3e944b845bc62a445a2c2f69
Successfully built gcloud


In [ ]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=AbdsCdabXF9f2CueApvFSt7XydNgUX&prompt=consent&token_usage=remote&access_type=offline&code_challenge=V59B_bjbxeZT9T0Qp-Aq0ritKdJTvlJpgflR1-mWCaI&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AQlEd8zLS-cgQMgrvNdo8Yl--Doj4k-_EWifhnTEuT8CernNSTx6j8oGF9R5Eiu5sFLqTg

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [ ]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import csv
import re

In [ ]:
# Cleaning workspace
# No direct equivalent for `rm(list = ls())` and `gc()` in Python,
# but garbage collection can be done using `gc` module if needed.

# Load the data
pep = pd.read_excel('Carreiras GF - d248cf93e92f4e17ba721ec734c69b4d.xlsx', sheet_name='Document_CH1035')
pep.columns = pep.columns.str.lower().str.replace(' ', '_')

In [ ]:
pep

,órgão_(sigla),qtd._cargos_ocupados,qtd._cargos_aprovados,qtd._cargos_distribuidos,agrupamentos_de_cargos_2,agrupamentos_de_cargos_1,grupo-cargo_(sigla),cargo_(com_código),escolaridade_do_cargo,cargo_em_extinção_(indicador),subsídio_(indicador),plano/carreira,nome_do_cargo,código_do_cargo,qtd._cargos_vagos,remuneração_-_mínimo_valor_inicial\n,remuneração_-_máximo_valor_final
0,CGU,1821.0,3055.0,2100.0,Grupo de Gestão - (Carreira),Auditor Federal de Finanças e Controle - NS (C...,G,Auditor Federal De Financas E Controle (403100),NS,0.0,S,Sem Plano,AUDITOR FEDERAL DE FINANCAS E CONTROLE,403100.0,1234.0,20924.8,29832.94
1,CGU,317.0,1960.0,546.0,Grupo de Gestão - (Carreira),Técnico Federal de Finanças e Controle - NI (C...,G,Tecnico Federal De Financas E Controle (403101),NM,0.0,S,Sem Plano,TECNICO FEDERAL DE FINANCAS E CONTROLE,403101.0,1643.0,7938.81,13640.89
2,CGU,6.0,21.0,10.0,PGPE - Plano Geral de Cargos do Poder Executiv...,Cargos de Nível Intermediário do PGPE - NI,PGPE,Agente Administrativo (481004),NM,0.0,N,Plano Geral de Cargos do Poder Executivo - PGPE,AGENTE ADMINISTRATIVO,481004.0,15.0,4183.03,4920.3
3,CGU,5.0,5.0,5.0,PGPE - Plano Geral de Cargos do Poder Executiv...,Cargos de Nível Intermediário do PGPE - NI,PGPE,Motorista Oficial (481262),NM,1.0,N,Plano Geral de Cargos do Poder Executivo - PGPE,MOTORISTA OFICIAL,481262.0,0.0,4183.03,4920.3
4,CGU,2.0,2.0,2.0,PGPE - Plano Geral de Cargos do Poder Executiv...,Cargos de Nível Intermediário do PGPE - NI,PGPE,Agente De Portaria (481026),NM,1.0,N,Plano Geral de Cargos do Poder Executivo - PGPE,AGENTE DE PORTARIA,481026.0,0.0,4183.03,4920.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4534,SIPEC,0.0,1.0,0.0,Tecnologia Militar - (Plano de Carreiras e Car...,Técnico de Tecnologia Militar - NI (Carreira d...,CST,Tecnico De Tecnologia Militar (443001),NM,0.0,N,Plano de Carreira dos Cargos de Tecnologia Mil...,TECNICO DE TECNOLOGIA MILITAR,443001.0,1.0,4187.85,9280.53
4535,SIPEC,0.0,2.0,0.0,-,-,-,-,-,0.0,-,-,-,700001.0,2.0,-,-
4536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4537,Selection Status:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pep.keys()

Index(['órgão_(sigla)', 'qtd._cargos_ocupados', 'qtd._cargos_aprovados',
       'qtd._cargos_distribuidos', 'agrupamentos_de_cargos_2',
       'agrupamentos_de_cargos_1', 'grupo-cargo_(sigla)', 'cargo_(com_código)',
       'escolaridade_do_cargo', 'cargo_em_extinção_(indicador)',
       'subsídio_(indicador)', 'plano/carreira', 'nome_do_cargo',
       'código_do_cargo', 'qtd._cargos_vagos',
       'remuneração_-_mínimo_valor_inicial\n',
       'remuneração_-_máximo_valor_final'],
      dtype='object')

In [ ]:
# Filter the data
pep = pep.iloc[:-3]
pep = pep[pep['cargo_em_extinção_(indicador)'] == 0]


In [ ]:
pep.dropna()

,órgão_(sigla),qtd._cargos_ocupados,qtd._cargos_aprovados,qtd._cargos_distribuidos,agrupamentos_de_cargos_2,agrupamentos_de_cargos_1,grupo-cargo_(sigla),cargo_(com_código),escolaridade_do_cargo,cargo_em_extinção_(indicador),subsídio_(indicador),plano/carreira,nome_do_cargo,código_do_cargo,qtd._cargos_vagos,remuneração_-_mínimo_valor_inicial\n,remuneração_-_máximo_valor_final
0,CGU,1821.0,3055.0,2100.0,Grupo de Gestão - (Carreira),Auditor Federal de Finanças e Controle - NS (C...,G,Auditor Federal De Financas E Controle (403100),NS,0.0,S,Sem Plano,AUDITOR FEDERAL DE FINANCAS E CONTROLE,403100.0,1234.0,20924.8,29832.94
1,CGU,317.0,1960.0,546.0,Grupo de Gestão - (Carreira),Técnico Federal de Finanças e Controle - NI (C...,G,Tecnico Federal De Financas E Controle (403101),NM,0.0,S,Sem Plano,TECNICO FEDERAL DE FINANCAS E CONTROLE,403101.0,1643.0,7938.81,13640.89
2,CGU,6.0,21.0,10.0,PGPE - Plano Geral de Cargos do Poder Executiv...,Cargos de Nível Intermediário do PGPE - NI,PGPE,Agente Administrativo (481004),NM,0.0,N,Plano Geral de Cargos do Poder Executivo - PGPE,AGENTE ADMINISTRATIVO,481004.0,15.0,4183.03,4920.3
5,CGU,1.0,6.0,2.0,PGPE - Plano Geral de Cargos do Poder Executiv...,Cargos de Nível Intermediário do PGPE - NI,PGPE,Tecnico De Contabilidade (481334),NM,0.0,N,Plano Geral de Cargos do Poder Executivo - PGPE,TECNICO DE CONTABILIDADE,481334.0,5.0,4183.03,4920.3
7,CGU,1.0,12.0,2.0,PGPE - Plano Geral de Cargos do Poder Executiv...,Cargos de Nível Superior do PGPE - NS,PGPE,Administrador (480002),NS,0.0,N,Plano Geral de Cargos do Poder Executivo - PGPE,ADMINISTRADOR,480002.0,11.0,6255.9,9728.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4531,SIPEC,0.0,1.0,0.0,Seguridade Social e do Trabalho - (Carreira),Cargos de Nível Superior - NS (Carreira da Seg...,CSST,Tecnico Em Assunto Educacionais (430092),NS,0.0,N,Sem Plano,TECNICO EM ASSUNTO EDUCACIONAIS,430092.0,1.0,6277.88,9699.9
4532,SIPEC,0.0,10.0,0.0,Seguro Social - (Carreira),Analista do Seguro Social - NS (Carreira do Se...,CSS,Analista Do Seguro Social (434500),NS,0.0,N,Sem Plano,ANALISTA DO SEGURO SOCIAL,434500.0,10.0,6831.11,14207.2
4533,SIPEC,0.0,133.0,0.0,Seguro Social - (Carreira),Técnico do Seguro Social - NI (Carreira do Seg...,CSS,Tecnico Do Seguro Social (434550),NM,0.0,N,Sem Plano,TECNICO DO SEGURO SOCIAL,434550.0,133.0,4454.71,9918.12
4534,SIPEC,0.0,1.0,0.0,Tecnologia Militar - (Plano de Carreiras e Car...,Técnico de Tecnologia Militar - NI (Carreira d...,CST,Tecnico De Tecnologia Militar (443001),NM,0.0,N,Plano de Carreira dos Cargos de Tecnologia Mil...,TECNICO DE TECNOLOGIA MILITAR,443001.0,1.0,4187.85,9280.53


In [ ]:
# Calculate the quantity and percentage of plans/careers by range of number of positions
indicador = pep[['agrupamentos_de_cargos_1', 'cargo_(com_código)']].drop_duplicates()
indicador

,agrupamentos_de_cargos_1,cargo_(com_código)
0,Auditor Federal de Finanças e Controle - NS (C...,Auditor Federal De Financas E Controle (403100)
1,Técnico Federal de Finanças e Controle - NI (C...,Tecnico Federal De Financas E Controle (403101)
2,Cargos de Nível Intermediário do PGPE - NI,Agente Administrativo (481004)
5,Cargos de Nível Intermediário do PGPE - NI,Tecnico De Contabilidade (481334)
7,Cargos de Nível Superior do PGPE - NS,Administrador (480002)
...,...,...
4515,Cargos de Nível Auxiliar - NA (Carreira da Seg...,Piloto De Lancha (430476)
4520,Cargos de Nível Intermediário - NI (Carreira d...,Laboratorista (430207)
4521,Cargos de Nível Intermediário - NI (Carreira d...,Auxiliar De Administracao (430171)
4528,Cargos de Nível Intermediário - NI (Carreira d...,Telefonista 30 Horas (430044)


In [ ]:
indicador = indicador.groupby('agrupamentos_de_cargos_1').size().reset_index(name='n')
indicador['faixa_cargos'] = pd.cut(indicador['n'], bins=[0, 1, 10, 40, 99, float('inf')],
                                   labels=['1', '2 a 10', '11 a 40', '41 a 99', 'Acima de 100'])
indicador

,agrupamentos_de_cargos_1,n,faixa_cargos
0,-,1,1
1,Advogado da União - Lotado na AGU - NS (Carrei...,1,1
2,Agente Executivo da CVM - NI (Carreira de Agen...,1,1
3,Agente Executivo da CVM/SUSEP - NI (Carreira d...,1,1
4,Agente Executivo da SUSEP - NI (Carreira de Ag...,1,1
...,...,...,...
246,Técnico em Regulação de Serviços Públicos de T...,1,1
247,Técnico em Regulação de Serviços de Transporte...,1,1
248,Técnico em Regulação de Serviços de Transporte...,1,1
249,Técnico em Regulação e Vigilância Sanitária - ...,1,1


In [ ]:
indicador1 = indicador.groupby('faixa_cargos').size().reset_index(name='qtde_planos')
indicador1

<ipython-input-17-c2391e5719d4>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  indicador1 = indicador.groupby('faixa_cargos').size().reset_index(name='qtde_planos')


,faixa_cargos,qtde_planos
0,1,160
1,2 a 10,65
2,11 a 40,21
3,41 a 99,5
4,Acima de 100,0


In [ ]:
indicador1['perc_planos'] = indicador1['qtde_planos'] / indicador1['qtde_planos'].sum() * 100
indicador1

,faixa_cargos,qtde_planos,perc_planos
0,1,160,63.745020
1,2 a 10,65,25.896414
2,11 a 40,21,8.366534
3,41 a 99,5,1.992032
4,Acima de 100,0,0.000000


In [ ]:
indicador.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   agrupamentos_de_cargos_1  251 non-null    object  
 1   n                         251 non-null    int64   
 2   faixa_cargos              251 non-null    category
dtypes: category(1), int64(1), object(1)
memory usage: 4.5+ KB


In [ ]:
indicador.rename(columns={'agrupamentos_de_cargos_1': 'agrupamentos'}, inplace=True)

In [ ]:
indicador.rename(columns={'n': 'quantidade_cargos'}, inplace=True)

In [ ]:
indicador.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   agrupamentos       251 non-null    object  
 1   quantidade_cargos  251 non-null    int64   
 2   faixa_cargos       251 non-null    category
dtypes: category(1), int64(1), object(1)
memory usage: 4.5+ KB


In [ ]:
indicador['faixa_cargos'].unique()

['1', '2 a 10', '11 a 40', '41 a 99']
Categories (5, object): ['1' < '2 a 10' < '11 a 40' < '41 a 99' < 'Acima de 100']

In [ ]:
indicador.head()

,agrupamentos,quantidade_cargos,faixa_cargos
0,-,1,1
1,Advogado da União - Lotado na AGU - NS (Carrei...,1,1
2,Agente Executivo da CVM - NI (Carreira de Agen...,1,1
3,Agente Executivo da CVM/SUSEP - NI (Carreira d...,1,1
4,Agente Executivo da SUSEP - NI (Carreira de Ag...,1,1


In [ ]:
indicador['quantidade_cargos'] = 1

In [ ]:
indicador

,agrupamentos,quantidade_cargos,faixa_cargos
0,-,1,1
1,Advogado da União - Lotado na AGU - NS (Carrei...,1,1
2,Agente Executivo da CVM - NI (Carreira de Agen...,1,1
3,Agente Executivo da CVM/SUSEP - NI (Carreira d...,1,1
4,Agente Executivo da SUSEP - NI (Carreira de Ag...,1,1
...,...,...,...
246,Técnico em Regulação de Serviços Públicos de T...,1,1
247,Técnico em Regulação de Serviços de Transporte...,1,1
248,Técnico em Regulação de Serviços de Transporte...,1,1
249,Técnico em Regulação e Vigilância Sanitária - ...,1,1


In [ ]:
indicador['quantidade_cargos'].unique()

array([1])

In [ ]:
indicador.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   agrupamentos       251 non-null    object  
 1   quantidade_cargos  251 non-null    int64   
 2   faixa_cargos       251 non-null    category
dtypes: category(1), int64(1), object(1)
memory usage: 4.5+ KB


In [ ]:
schema = [
    bigquery.SchemaField('agrupamentos', 'STRING', description='Faixa de cargos por plano'),
    bigquery.SchemaField('quantidade_cargos', 'INTEGER', description='Quantidade de cargos por plano'),
    bigquery.SchemaField('faixa_cargos', 'STRING', description='Faixa de cargos por plano')
]


In [ ]:
## Subindo para datalake
client = bigquery.Client(project='repositoriodedadosgpsp')
dataset_ref = client.dataset('atracao_pre_selecao')

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
table_ref = dataset_ref.table('PEP_faixa_cargos_v3') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(indicador, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=27ac4994-4f03-4e16-be42-bf98d288da9b>

dados vieram certos do spreadsheet
consigo fazer gráfico de barra com essa tabela sem problemas